In [1]:
!pip install bitsandbytes accelerate
!pip install wandb
!pip install loguru
# !pip install liger-kernel
!pip install peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.8 MB/s eta 0:00:00


In [2]:
from kaggle_secrets import UserSecretsClient
import wandb
import huggingface_hub

user_secrets = UserSecretsClient()
secret_value_wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
secret_value_hf_api_key = user_secrets.get_secret("Shirsh_Main_HF")

huggingface_hub.login(secret_value_hf_api_key)
wandb.login(key=secret_value_wandb_api_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from loguru import logger
from tqdm import tqdm

In [4]:
dataset = datasets.load_dataset("shirsh10mall/PII_Detection_Processed_Data_Text_Generation")
dataset

README.md:   0%|          | 0.00/560 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.49M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/390k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4890 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1240 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source_text', 'uid', 'masked_text', 'source_text_token_length', 'target_text'],
        num_rows: 4890
    })
    validation: Dataset({
        features: ['source_text', 'uid', 'masked_text', 'source_text_token_length', 'target_text'],
        num_rows: 1240
    })
})

In [5]:
label_mapper = {
    'EMAIL': 'Email',
    'IDCARDNUM': 'ID_Card_Number',
    'TAXNUM': 'Tax_Number',
    'CREDITCARDNUMBER': 'Credit_Card_Number',
    'BUILDINGNUM': 'Building_Number',
    'TELEPHONENUM': 'Telephone_Number',
    'USERNAME': 'Username',
    'STREET': 'Street_Address',
    'ZIPCODE': 'Zip_Code',
    'GIVENNAME': 'First_Name',
    'ACCOUNTNUM': 'Account_Number',
    'SOCIALNUM': 'Social_Security_Number',
    'CITY': 'City',
    'DATEOFBIRTH': 'Date_of_Birth',
    'SURNAME': 'Last_Name',
    'PASSWORD': 'Password',
    'DRIVERLICENSENUM': 'Drivers_License_Number'
}

labels_list = list(label_mapper.values())
labels_list

['Email',
 'ID_Card_Number',
 'Tax_Number',
 'Credit_Card_Number',
 'Building_Number',
 'Telephone_Number',
 'Username',
 'Street_Address',
 'Zip_Code',
 'First_Name',
 'Account_Number',
 'Social_Security_Number',
 'City',
 'Date_of_Birth',
 'Last_Name',
 'Password',
 'Drivers_License_Number']

In [6]:
model_id = "meta-llama/Llama-3.2-1B"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [13]:
# Prompt
prompt = """
Please analyze the provided text and identify any Personally Identifiable Information (PII) that falls under the following categories:
{list_of_labels}

After identifying the PII, rewrite the text by masking each detected PII entity in square brackets. Label each masked entity with its corresponding PII category. Ensure that the rest of the text remains exactly the same, with no alterations.


Input Text:
{input_text}


Output Text: 
{output_text}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def data_formatter(examples):
    input_text_list = examples["source_text"]
    output_text_list = examples["target_text"]
    texts = []
    for input_text, output_text in zip(input_text_list, output_text_list):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = tokenizer.bos_token + prompt.format(list_of_labels=labels_list, input_text=input_text, output_text=output_text) + EOS_TOKEN
        texts.append(text)
        
    token_length_list = tokenizer(texts, return_attention_mask=False, return_length=True)["length"]
    batch_max_length = max(token_length_list)
        
    final_text_list = []
    for text, length in zip(texts, token_length_list):
        text = text + (batch_max_length - length)*tokenizer.eos_token
        final_text_list.append(text)
    return final_text_list # {"text": final_text_list, "output_text_list": output_text_list}

output = data_formatter(dataset["train"][3:6])

In [8]:
# Quantization Configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # NF4 for better quantization performance
    bnb_4bit_compute_dtype="float16",  # Better computation speed for bfloat16
    bnb_4bit_use_double_quant=True  # Nested quantization to further reduce memory usage
)

# Peft-Lora Layer
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    use_rslora=True,
    use_dora=True,
    bias="lora_only",
    task_type="CAUSAL_LM",
    # target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
    # init_lora_weights
)

In [9]:
# Multi-GPU usinf accelarate - https://huggingface.co/docs/trl/en/sft_trainer#multi-gpu-training

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)


model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# print(model.get_memory_footprint())
model

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict(
                  (default): lora.dora.D

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Decode predictions and labels from token IDs to text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU Metric
    bleu_result = bleu.compute(predictions=[pred.split() for pred in decoded_preds],
                               references=[[label.split()] for label in decoded_labels])

    # ROUGE Metric
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # METEOR Metric
    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)

    # Prepare the results dictionary
    result = {
        "bleu": bleu_result['bleu'],
        "rouge1": rouge_result["rouge1"].mid.fmeasure,
        "rouge2": rouge_result["rouge2"].mid.fmeasure,
        "rougeL": rouge_result["rougeL"].mid.fmeasure,
        "meteor": meteor_result["meteor"]
    }
    
    return result


In [11]:
training_arguments = SFTConfig(
    output_dir="./training_results",
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    eval_on_start=True,
    batch_eval_metrics=True,
    save_total_limit=1,
    
    group_by_length=True,
    max_seq_length=512,
    dataset_text_field="text",
    packing=True,
    # dataset_num_proc = 2,
    
    # auto_find_batch_size=False,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    dataloader_num_workers=2,
    
    torch_empty_cache_steps=16,
    optim="paged_adamw_32bit",
    max_grad_norm=0.3,
    max_steps=-1,
    learning_rate=2e-4,
    weight_decay=0.001,
    
    gradient_checkpointing=False,
    gradient_accumulation_steps=1,
    eval_accumulation_steps=4,
    
    logging_steps=25,
    log_level="info",
    logging_nan_inf_filter=False,
    
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    
    load_best_model_at_end=True,
    metric_for_best_model="loss",

    report_to="wandb",
    
    # neftune_noise_alpha # https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments.neftune_noise_alpha
)

In [14]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    peft_config=peft_config,
    
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    
    formatting_func=data_formatter,
    
    compute_metrics=compute_metrics,
    args=training_arguments,
    # use_liger=True # https://huggingface.co/docs/trl/en/sft_trainer#liger-kernel-increase-20-throughput-and-reduces-60-memory-for-multi-gpu-training
)

Generating train split: 0 examples [00:00, ? examples/s]

ValueError: Error occurred while packing the dataset. Make sure that your dataset has enough samples to at least yield one packed sequence.

In [ ]:
trainer.train()

In [ ]:
model = trainer.model.merge_and_un